In [1]:
#Imports
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [2]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
Western Carolina University 
last updated: 2020-02-22 

pandas 1.0.1


In [3]:
# Create Backups
#df_backup = pd.read_csv('./Updates/STG_ZLLW_County_MedianListingPrice_AllHomes.txt')
#df_backup.to_csv('./Backups/STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP.txt')

In [4]:
#Load Land data
df_msp = pd.read_csv('http://files.zillowstatic.com/research/public/County/Sale_Prices_County.csv',
                     encoding='ISO-8859-1')

#Display table to ensure data loaded correctly
df_msp.head()

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
0,3101,Los Angeles County,California,1,458300.0,448000.0,431700.0,419900.0,403800.0,388700.0,...,605500.0,604800.0,603500.0,606200.0,608700.0,612900.0,611200.0,615700.0,624600.0,633000.0
1,139,Cook County,Illinois,2,285300.0,280300.0,255900.0,242800.0,235800.0,234900.0,...,256200.0,245300.0,236200.0,232100.0,232700.0,234200.0,235400.0,237300.0,241400.0,244700.0
2,1090,Harris County,Texas,3,137600.0,135500.0,132000.0,132000.0,132400.0,134300.0,...,207700.0,208500.0,210000.0,209600.0,211300.0,213000.0,218700.0,220600.0,222700.0,222600.0
3,2402,Maricopa County,Arizona,4,231000.0,221400.0,212100.0,207500.0,201500.0,196100.0,...,249500.0,249500.0,251200.0,254800.0,255900.0,257900.0,260700.0,265700.0,269100.0,275000.0
4,2841,San Diego County,California,5,401900.0,395400.0,380800.0,369700.0,353500.0,341600.0,...,561200.0,559800.0,563100.0,561500.0,565800.0,562800.0,569400.0,576700.0,583600.0,587700.0


In [9]:
df_msp = df_msp.drop(columns = ['RegionID', 'StateName'], axis = 1)
df_msp

,RegionName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,2008-09,2008-10,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
0,Los Angeles County,1,458300.0,448000.0,431700.0,419900.0,403800.0,388700.0,373000.0,359600.0,...,605500.0,604800.0,603500.0,606200.0,608700.0,612900.0,611200.0,615700.0,624600.0,633000.0
1,Cook County,2,285300.0,280300.0,255900.0,242800.0,235800.0,234900.0,238300.0,238300.0,...,256200.0,245300.0,236200.0,232100.0,232700.0,234200.0,235400.0,237300.0,241400.0,244700.0
2,Harris County,3,137600.0,135500.0,132000.0,132000.0,132400.0,134300.0,135700.0,135200.0,...,207700.0,208500.0,210000.0,209600.0,211300.0,213000.0,218700.0,220600.0,222700.0,222600.0
3,Maricopa County,4,231000.0,221400.0,212100.0,207500.0,201500.0,196100.0,186900.0,179200.0,...,249500.0,249500.0,251200.0,254800.0,255900.0,257900.0,260700.0,265700.0,269100.0,275000.0
4,San Diego County,5,401900.0,395400.0,380800.0,369700.0,353500.0,341600.0,331200.0,321200.0,...,561200.0,559800.0,563100.0,561500.0,565800.0,562800.0,569400.0,576700.0,583600.0,587700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,Camden County,1263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,218500.0,224900.0,231700.0,233400.0,238500.0,224100.0,242200.0,259700.0,276300.0,258000.0
1263,Valley County,1264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,370100.0,385500.0,378600.0,380500.0,348900.0,355900.0,368800.0,405100.0,409200.0,411200.0
1264,Clear Creek County,1265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,343400.0,410400.0,383300.0,380100.0,359200.0,377100.0,378100.0,401000.0,425100.0,490700.0
1265,Gilpin County,1266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,370500.0,380800.0,408900.0,422600.0,398700.0,361100.0,350800.0,365500.0,363000.0,NaN


In [5]:
#Filter data to NC
filter1 = df_msp['State'] == "NC"
df_msp_nc = df_msp[filter1]

#Check to ensure filter worked
df_msp_nc.head(5)

,RegionID,RegionName,StateName,SizeRank,2008-03,2008-04,2008-05,2008-06,2008-07,2008-08,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
44,3102,Mecklenburg County,North Carolina,45,164100.0,160200.0,156300.0,153600.0,153200.0,151500.0,...,241200.0,241300.0,241700.0,243400.0,241100.0,246700.0,248900.0,254900.0,260600.0,267900.0
51,324,Wake County,North Carolina,52,186500.0,182500.0,180600.0,180400.0,182500.0,182100.0,...,277500.0,272300.0,271200.0,273300.0,278500.0,280800.0,282700.0,284000.0,288200.0,292400.0
127,2312,Guilford County,North Carolina,128,140100.0,135700.0,128900.0,125700.0,128300.0,127800.0,...,163800.0,162600.0,162400.0,164100.0,160300.0,160300.0,159000.0,162300.0,166500.0,168600.0
175,1712,Forsyth County,North Carolina,176,128200.0,127300.0,124700.0,127700.0,129900.0,131300.0,...,149300.0,153500.0,156000.0,162600.0,167000.0,165100.0,163000.0,167900.0,171500.0,171000.0
188,2912,Cumberland County,North Carolina,189,NaN,NaN,NaN,NaN,NaN,NaN,...,113200.0,117800.0,121800.0,124500.0,121300.0,122300.0,125400.0,127000.0,131500.0,130900.0


In [6]:
#View data types of dataframe
df_msp_nc.dtypes

RegionID        int64
RegionName     object
StateName      object
SizeRank        int64
2008-03       float64
               ...   
2019-09       float64
2019-10       float64
2019-11       float64
2019-12       float64
2020-01       float64
Length: 147, dtype: object

In [7]:
#Change MunicipalCodeFIPS dtype to add leading 0's
df_msp_nc.loc[ :, 'MunicipalCodeFIPS'] = df_msp_nc['MunicipalCodeFIPS'].astype(str)
df_msp_nc.dtypes

KeyError: 'MunicipalCodeFIPS'

In [ ]:
#Add leading 0's and check to ensure they were added
df_msp_nc.loc[ :, 'MunicipalCodeFIPS'] = df_msp_nc['MunicipalCodeFIPS'].str.zfill(3)
df_msp_nc.head(5)

In [ ]:
# Set Index to Region Name
df_msp_nc.set_index(df_msp_nc['RegionName'], inplace = True)
df_msp_nc

In [ ]:
# Drop Region Name column
df_msp_nc.drop('RegionName', axis = 1, inplace = True)
df_msp_nc

In [ ]:
#Save to csv file for export in Excel
#df_msp_nc.to_csv('./Updates/STG_ZLLW_County_MedianListingPrice_AllHomes.txt', sep ='\t')

In [ ]:
#Reset Index for upload to database
df_msp_nc = df_msp_nc.reset_index()    

In [ ]:
#Fill NaN values for upload to database
df_msp_nc['Metro'] = df_msp_nc['Metro'].replace(np.nan,'', regex=True)

column_list = df_msp_nc.columns.values
for i in column_list:
    df_msp_nc.loc[df_msp_nc[i].isnull(),i]=0

In [ ]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;',
                    autocommit=True)

c = con.cursor()

In [ ]:
#Drop old backup table
c.execute('drop table STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP')

In [ ]:
#Create new backup
c.execute('''sp_rename 'dbo.STG_ZLLW_County_MedianListingPrice_AllHomes','STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP';''')

In [ ]:
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_ZLLW_County_MedianListingPrice_AllHomes](
	[RegionName] [varchar](40) NULL,
	[State] [varchar](2) NULL,
	[Metro] [varchar](40) NULL,
	[StateCodeFIPS] [varchar](2) NULL,
	[MunicipalCodeFIPS] [varchar](3) NULL,
	[SizeRank] [smallint] NULL,
	[2010-01] [float] NULL,
	[2010-02] [float] NULL,
	[2010-03] [float] NULL,
	[2010-04] [float] NULL,
	[2010-05] [float] NULL,
	[2010-06] [float] NULL,
	[2010-07] [float] NULL,
	[2010-08] [float] NULL,
	[2010-09] [float] NULL,
	[2010-10] [float] NULL,
	[2010-11] [float] NULL,
	[2010-12] [float] NULL,
	[2011-01] [float] NULL,
	[2011-02] [float] NULL,
	[2011-03] [float] NULL,
	[2011-04] [float] NULL,
	[2011-05] [float] NULL,
	[2011-06] [float] NULL,
	[2011-07] [float] NULL,
	[2011-08] [float] NULL,
	[2011-09] [float] NULL,
	[2011-10] [float] NULL,
	[2011-11] [float] NULL,
	[2011-12] [float] NULL,
	[2012-01] [float] NULL,
	[2012-02] [float] NULL,
	[2012-03] [float] NULL,
	[2012-04] [float] NULL,
	[2012-05] [float] NULL,
	[2012-06] [float] NULL,
	[2012-07] [float] NULL,
	[2012-08] [float] NULL,
	[2012-09] [float] NULL,
	[2012-10] [float] NULL,
	[2012-11] [float] NULL,
	[2012-12] [float] NULL,
	[2013-01] [float] NULL,
	[2013-02] [float] NULL,
	[2013-03] [float] NULL,
	[2013-04] [float] NULL,
	[2013-05] [float] NULL,
	[2013-06] [float] NULL,
	[2013-07] [float] NULL,
	[2013-08] [float] NULL,
	[2013-09] [float] NULL,
	[2013-10] [float] NULL,
	[2013-11] [float] NULL,
	[2013-12] [float] NULL,
	[2014-01] [float] NULL,
	[2014-02] [float] NULL,
	[2014-03] [float] NULL,
	[2014-04] [float] NULL,
	[2014-05] [float] NULL,
	[2014-06] [float] NULL,
	[2014-07] [float] NULL,
	[2014-08] [float] NULL,
	[2014-09] [float] NULL,
	[2014-10] [float] NULL,
	[2014-11] [float] NULL,
	[2014-12] [float] NULL,
	[2015-01] [float] NULL,
	[2015-02] [float] NULL,
	[2015-03] [float] NULL,
	[2015-04] [float] NULL,
	[2015-05] [float] NULL,
	[2015-06] [float] NULL,
	[2015-07] [float] NULL,
	[2015-08] [float] NULL,
	[2015-09] [float] NULL,
	[2015-10] [float] NULL,
	[2015-11] [float] NULL,
	[2015-12] [float] NULL,
	[2016-01] [float] NULL,
	[2016-02] [float] NULL,
	[2016-03] [float] NULL,
	[2016-04] [float] NULL,
	[2016-05] [float] NULL,
	[2016-06] [float] NULL,
	[2016-07] [float] NULL,
	[2016-08] [float] NULL,
	[2016-09] [float] NULL,
	[2016-10] [float] NULL,
	[2016-11] [float] NULL,
	[2016-12] [float] NULL,
	[2017-01] [float] NULL,
	[2017-02] [float] NULL,
	[2017-03] [float] NULL,
	[2017-04] [float] NULL,
	[2017-05] [float] NULL,
	[2017-06] [float] NULL,
	[2017-07] [float] NULL,
	[2017-08] [float] NULL,
	[2017-09] [float] NULL,
	[2017-10] [float] NULL,
	[2017-11] [float] NULL,
	[2017-12] [float] NULL,
	[2018-01] [float] NULL,
	[2018-02] [float] NULL,
	[2018-03] [float] NULL,
	[2018-04] [float] NULL,
	[2018-05] [float] NULL,
	[2018-06] [float] NULL,
	[2018-07] [float] NULL,
	[2018-08] [float] NULL,
	[2018-09] [float] NULL,
	[2018-10] [float] NULL,
	[2018-11] [float] NULL,
	[2018-12] [float] NULL,
	[2019-01] [float] NULL,
	[2019-02] [float] NULL,
	[2019-03] [float] NULL,
	[2019-04] [float] NULL,
	[2019-05] [float] NULL,
	[2019-06] [float] NULL,
	[2019-07] [float] NULL,
	[2019-08] [float] NULL,
	[2019-09] [float] NULL,
	[2019-10] [float] NULL,
	[2019-11] [float] NULL,
	[2019-12] [float] NULL,
    [2020-01] [float] NULL,
    [2020-02] [float] NULL,
    [2020-03] [float] NULL,
    [2020-04] [float] NULL,
    [2020-05] [float] NULL,
    [2020-06] [float] NULL,
    [2020-07] [float] NULL,
    [2020-08] [float] NULL,
    [2020-09] [float] NULL,
    [2020-10] [float] NULL,
    [2020-11] [float] NULL,
    [2020-12] [float] NULL,
    [2021-01] [float] NULL,
    [2021-02] [float] NULL,
    [2021-03] [float] NULL,
    [2021-04] [float] NULL,
    [2021-05] [float] NULL,
    [2021-06] [float] NULL
    [2021-07] [float] NULL,
    [2021-08] [float] NULL,
    [2021-09] [float] NULL,
    [2021-10] [float] NULL,
    [2021-11] [float] NULL,
    [2021-12] [float] NULL,
    [2022-01] [float] NULL,
    [2022-02] [float] NULL,
    [2022-03] [float] NULL,
    [2022-04] [float] NULL,
    [2022-05] [float] NULL,
    [2022-06] [float] NULL,
    [2022-07] [float] NULL,
    [2022-08] [float] NULL,
    [2022-09] [float] NULL,
    [2022-10] [float] NULL,
    [2022-11] [float] NULL,
    [2022-12] [float] NULL
) ON [PRIMARY]''')

In [ ]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#warning: discard old table if exists
df_msp_nc.to_sql('STG_ZLLW_County_MedianListingPrice_AllHomes', con=engine, if_exists='replace', index=False)